# A simple neural network for cats vs dogs classification
https://www.pyimagesearch.com/2016/09/26/a-simple-neural-network-with-python-and-keras/

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from imutils import paths
import numpy as np
import cv2
import os

Using TensorFlow backend.


## Function definition

In [2]:
def image_to_feature_vector(image, size=(32, 32)):
    # Resize the image to a fixed size, then flatten the image into a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

## Configuration

In [3]:
dataset_path = 'kaggle_dogs_vs_cats/train/'
test_path = 'test_images/'
output_model_path = 'output/simple_neural_network.hdf5'

## Prepare dataset

In [4]:
# Grab the list of images that we'll be describing
imagePaths = list(paths.list_images(dataset_path))

# initialize the data matrix and labels list
data = []
labels = []

# Loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # Load the image and extract the class label (assuming that our
    # path has the format: /path/to/dataset/{class}.{image_num}.jpg
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
    # Construct a feature vector raw pixel intensities, then update the data matrix and labels list
    features = image_to_feature_vector(image)
    data.append(features)
    labels.append(label)
    
    # Show an update every 1,000 images
    if i > 0 and i % 1000 == 0:
        print("Processed {}/{}".format(i, len(imagePaths)))

Processed 1000/25000
Processed 2000/25000
Processed 3000/25000
Processed 4000/25000
Processed 5000/25000
Processed 6000/25000
Processed 7000/25000
Processed 8000/25000
Processed 9000/25000
Processed 10000/25000
Processed 11000/25000
Processed 12000/25000
Processed 13000/25000
Processed 14000/25000
Processed 15000/25000
Processed 16000/25000
Processed 17000/25000
Processed 18000/25000
Processed 19000/25000
Processed 20000/25000
Processed 21000/25000
Processed 22000/25000
Processed 23000/25000
Processed 24000/25000


In [5]:
# Encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Scale the input image pixels to the range [0, 1], then transform the labels into vectors
# in the range [0, num_classes] -- this generates a vector for each label where the index
# of the label is set to `1` and all other entries to `0`
data = np.array(data) / 255.0
labels = np_utils.to_categorical(labels, 2)

# Partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

## Network architecture

In [6]:
model = Sequential()
model.add(Dense(768, input_dim=3072, kernel_initializer="uniform", activation="relu"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(2))
model.add(Activation("softmax"))

## Training
Training using Stochastic Gradient Descent

In [7]:
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])  # Binary as there are only two classes
model.fit(trainData, trainLabels, epochs=50, batch_size=128, verbose=1)

Epoch 1/50
18750/18750 [==============================] - 1s 78us/step - loss: 0.6838 - accuracy: 0.5627
Epoch 2/50
18750/18750 [==============================] - 1s 73us/step - loss: 0.6637 - accuracy: 0.5946
Epoch 3/50
18750/18750 [==============================] - 1s 76us/step - loss: 0.6531 - accuracy: 0.6125
Epoch 4/50
18750/18750 [==============================] - 1s 76us/step - loss: 0.6465 - accuracy: 0.6229
Epoch 5/50
18750/18750 [==============================] - 1s 76us/step - loss: 0.6413 - accuracy: 0.6299
Epoch 6/50
18750/18750 [==============================] - 1s 77us/step - loss: 0.6350 - accuracy: 0.6394
Epoch 7/50
18750/18750 [==============================] - 1s 77us/step - loss: 0.6313 - accuracy: 0.6422
Epoch 8/50
18750/18750 [==============================] - ETA: 0s - loss: 0.6268 - accuracy: 0.64 - 1s 79us/step - loss: 0.6267 - accuracy: 0.6457
Epoch 9/50
18750/18750 [==============================] - 1s 78us/step - loss: 0.6206 - accuracy: 0.6520
Epoch 10/50
1

## Evaluation

In [8]:
# Show the accuracy on the testing set
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

# Dump the network architecture and weights to file
model.save(output_model_path)

6250/6250 [==============================] - 0s 35us/step
loss=0.5968, accuracy: 68.1120%


## Visualize the classification results

In [ ]:
# Initialize the class labels for the Kaggle dogs vs cats dataset
CLASSES = ["cat", "dog"]

# Loop over our testing images
for imagePath in paths.list_images(test_path):
    # Load the image, resize it to a fixed 32 x 32 pixels (ignoring aspect ratio),
    # and then extract features from it
    image = cv2.imread(imagePath)
    features = image_to_feature_vector(image) / 255.0
    features = np.array([features])
    
    # Classify the image using our extracted features and pre-trained neural network
    probs = model.predict(features)[0]
    prediction = probs.argmax(axis=0)
    
    # Draw the class and probability on the test image and display it on our screen
    label = "{}: {:.2f}%".format(CLASSES[prediction], probs[prediction] * 100)
    cv2.putText(image, label, (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
    cv2.imshow("Image", image)
    cv2.waitKey(0)